In [16]:
from main import buildBST, getGRCIndex
from models.definitions.GAT import GAT
from utils.constants import LayerType

import gzip
import json
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
import torch
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.colors as mcolors

In [6]:
with gzip.open('../xbar/1/xbar.json.gz','rb') as f:
    design = json.loads(f.read().decode('utf-8'))
    
instances = pd.DataFrame(design['instances'])
nets = pd.DataFrame(design['nets'])

conn=np.load('../xbar/1/xbar_connectivity.npz')
A = coo_matrix((conn['data'], (conn['row'], conn['col'])), shape=conn['shape'])
A = A.__mul__(A.T)

congestion_data = np.load('../xbar/1/xbar_congestion.npz')
xbst = buildBST(congestion_data['xBoundaryList'])
ybst = buildBST(congestion_data['yBoundaryList'])
demand = np.zeros(shape = [instances.shape[0],])
congestion = np.zeros(shape = [instances.shape[0],])
capacity = np.zeros(shape = [instances.shape[0],])

for k in range(instances.shape[0]):
    # print(k)
    xloc = instances.iloc[k]['xloc']; yloc = instances.iloc[k]['yloc']
    i,j=getGRCIndex(xloc,yloc,xbst,ybst)
    d = 0 
    c = 0
    for l in list(congestion_data['layerList']): 
        lyr=list(congestion_data['layerList']).index(l)
        d += congestion_data['demand'][lyr][i][j]
        c += congestion_data['capacity'][lyr][i][j]
    demand[k] = d
    congestion[k] = d - c
    capacity[k] = c

instances['routing_demand'] = demand
instances['congestion'] = congestion
instances['capacity'] = capacity

In [8]:
with gzip.open('../cells.json.gz','rb') as f:
    cells = json.loads(f.read().decode('utf-8'))

grc_pos = []
pin_count = []
for i in range(len(instances)):
    point = instances.loc[i]
    j = getGRCIndex(point.xloc,point.yloc,xbst,ybst)
    grc_pos.append(j)
    pin_count.append(len(cells[instances.loc[i].cell]['terms']))
instances['GRC_Index'] = grc_pos
instances['pin_count'] = pin_count
instances['pin_total'] = instances['pin_count'].groupby(instances['GRC_Index']).transform('sum')

In [10]:
instances

,name,id,xloc,yloc,cell,orient,routing_demand,congestion,capacity,GRC_Index,pin_count,pin_total
0,clk_gate_out_reg/latch,0,41984,44544,23,0,20.0,-8.0,28.0,"(29, 27)",4,12
1,clk_gate_out_reg_0/latch,1,41984,47616,23,6,23.0,-5.0,28.0,"(31, 27)",4,8
2,clk_gate_out_reg_1/latch,2,44160,44544,23,0,23.0,-10.0,33.0,"(29, 28)",4,10
3,clk_gate_out_reg_2/latch,3,44160,47616,23,0,22.0,-11.0,33.0,"(31, 28)",4,8
4,clk_gate_out_reg_3/latch,4,46336,47616,23,0,21.0,-2.0,23.0,"(31, 29)",4,4
...,...,...,...,...,...,...,...,...,...,...,...,...
3947,U4123,3947,21888,53760,42,4,31.0,0.0,31.0,"(35, 13)",3,12
3948,U4125,3948,33664,66048,42,0,30.0,0.0,30.0,"(43, 21)",3,16
3949,U4128,3949,23296,66048,34,0,27.0,-2.0,29.0,"(43, 14)",2,17
3950,ZCTSBUF_205_132,3950,40576,44544,11,0,28.0,-5.0,33.0,"(29, 26)",2,12


In [11]:
G = nx.from_numpy_array(A)

In [ ]:
sourcenodes, targetnodes = np.nonzero(A)
edgeindex = np.vstack((sourcenodes, targetnodes))
# part of the input for GAT

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [12]:
feature_data = instances[['xloc', 'yloc', 'cell', 'pin_total']]
in_nodes_features = torch.tensor(feature_data.values, dtype=torch.float32)

edges = torch.tensor(np.array(list(G.edges)), dtype=torch.int64)
topology = edges.t().contiguous()

labels = torch.tensor(instances['routing_demand'].values, dtype=torch.int64)

graph_data = (in_nodes_features, topology, labels)

In [13]:
graph_data

(tensor([[4.1984e+04, 4.4544e+04, 2.3000e+01, 1.2000e+01],
         [4.1984e+04, 4.7616e+04, 2.3000e+01, 8.0000e+00],
         [4.4160e+04, 4.4544e+04, 2.3000e+01, 1.0000e+01],
         ...,
         [2.3296e+04, 6.6048e+04, 3.4000e+01, 1.7000e+01],
         [4.0576e+04, 4.4544e+04, 1.1000e+01, 1.2000e+01],
         [4.6848e+04, 4.4544e+04, 1.1000e+01, 1.1000e+01]]),
 tensor([[   0,    0,    0,  ..., 3950, 3950, 3951],
         [3951, 3950,   11,  ..., 3951, 3950, 3951]]),
 tensor([20, 23, 23,  ..., 27, 28, 23]))